In [129]:
import pandas as pd
import numpy as np

In [130]:
year = '2022'

In [131]:
meas_ls = pd.read_csv('./seoul/서울시 대기오염 측정항목 정보.csv', encoding='euc-kr')
meas_loc = pd.read_csv('./seoul/서울시 대기오염 측정소 정보.csv', encoding='euc-kr')
meas = pd.read_csv(f'./seoul/AIR_HOUR_{year}.csv')
meas_items = meas_ls[['측정항목 코드', '측정항목 명(줄임 명칭)']]

In [132]:
# 측정소 코드를 기준으로 두 데이터 프레임을 병합합니다.
merged_df = pd.merge(meas, meas_loc[['측정소 코드', '측정소 이름']], on='측정소 코드', how='left')

# '측정항목' 열을 기준으로 두 데이터 프레임을 병합합니다.
merged_df = pd.merge(merged_df, meas_items, left_on='측정항목', right_on='측정항목 코드', how='left')

In [133]:
new_merged_df = merged_df.drop('측정항목 코드', axis=1)
new_merged_df

,측정일시,측정소 코드,측정항목,평균값,측정기 상태,국가 기준초과 구분,지자체 기준초과 구분,측정소 이름,측정항목 명(줄임 명칭)
0,2022010100,101,1,0.003,0,0,0,종로구,SO2
1,2022010100,101,3,0.016,0,0,0,종로구,NO2
2,2022010100,101,5,0.400,0,0,0,종로구,CO
3,2022010100,101,6,0.026,0,0,0,종로구,O3
4,2022010100,101,8,23.000,0,0,0,종로구,PM10
...,...,...,...,...,...,...,...,...,...
1313995,2022123123,125,3,0.054,0,0,0,강동구,NO2
1313996,2022123123,125,5,1.100,0,0,0,강동구,CO
1313997,2022123123,125,6,0.002,0,0,0,강동구,O3
1313998,2022123123,125,8,71.000,0,0,0,강동구,PM10


In [134]:
from datetime import datetime
new_merged_df['datetime'] = new_merged_df['측정일시'].apply(lambda x: datetime.strptime(str(x), "%Y%m%d%H"))
new_merged_df['날짜'] = new_merged_df['datetime'].dt.date
new_merged_df['시간'] = new_merged_df['datetime'].dt.hour

In [135]:
# '측정항목 명(줄임 명칭)'을 컬럼으로 올리기
pivot_df = new_merged_df.pivot(index=['측정일시', '측정소 코드', 'datetime', '날짜', '시간', '측정기 상태', '국가 기준초과 구분', '지자체 기준초과 구분', '측정소 이름'],
                    columns='측정항목 명(줄임 명칭)',
                    values='평균값').reset_index()

# 컬럼 이름 재설정
pivot_df.columns.name = None

# NaN 값 0으로 채우기
pivot_df = pivot_df.fillna(0)

# 결과 확인
pivot_df = pivot_df[['날짜', '시간', '측정소 이름', 'PM10', 'PM2.5', 'O3', 'SO2', 'NO2', 'CO']]

# 컬럼 이름 변경
new_column_names = {'측정소 이름': '구'}
pivot_df = pivot_df.rename(columns=new_column_names)

In [136]:
pivot_df

,날짜,시간,구,PM10,PM2.5,O3,SO2,NO2,CO
0,2022-01-01,0,종로구,23.0,12.0,0.026,0.003,0.016,0.4
1,2022-01-01,0,중구,25.0,10.0,0.016,0.003,0.024,0.4
2,2022-01-01,0,용산구,24.0,13.0,0.020,0.003,0.020,0.4
3,2022-01-01,0,은평구,19.0,10.0,0.019,0.004,0.019,0.8
4,2022-01-01,0,서대문구,23.0,13.0,0.019,0.000,0.020,0.7
...,...,...,...,...,...,...,...,...,...
250739,2022-12-31,23,강남구,49.0,0.0,0.003,0.004,0.053,0.9
250740,2022-12-31,23,강남구,0.0,36.0,0.000,0.000,0.000,0.0
250741,2022-12-31,23,송파구,47.0,34.0,0.002,0.003,0.051,0.9
250742,2022-12-31,23,강동구,71.0,0.0,0.002,0.003,0.054,1.1


In [137]:
pivot_df.to_csv(f'./ch_seoul/new_AIR_HOUR_{year}.csv', encoding='utf-8-sig', index=False)